In [1]:
import numpy as np
import skimage as sk
from aicspylibczi import CziFile
from glob import glob
import os
import napari

In [2]:
OIC_191_files = sorted(glob('OIC-191_Images/*.czi'))
OIC_192_files = sorted(glob('OIC-192_Images/*.czi'))

OIC_191_files_czi = list(map(CziFile,OIC_191_files))
OIC_192_files_czi = list(map(CziFile,OIC_192_files))

In [ ]:
def read_and_prep_images(czifile):
    scenes = [czifile.read_image(S=s)[0] for s in range(czifile.size[2])]
    squeezed = [np.squeeze(i) for i in scenes]
    average_projections = [np.round(np.mean(img,axis=1)).astype('uint16') for img in squeezed] #channel axis needs to be placed last for scikit-image
    
    return scenes, reorder, gauss_nuc, nuclei_channel 

In [18]:
test_img = OIC_191_files_czi[-1]

In [4]:
test_scenes = [test_img.read_image(S=s)[0] for s in range(test_img.size[2])]

In [24]:
test_scenes = [np.squeeze(scene) for scene in test_scenes]

In [8]:
test_scene.shape

(3, 26, 1012, 1012)

In [45]:
average_project = [np.round(np.mean(scene, axis=1)).astype('uint16') for scene in test_scenes]

In [48]:
padded_arrays = [np.pad(scene,pad_width=5,constant_values=(0,0)) for scene in average_project]

In [36]:
average_project = [np.moveaxis(scene,0,-1) for scene in average_project]

In [37]:
stack = np.stack(average_project)

In [52]:
montage_np = np.block(average_project)

In [53]:
montage_np.shape

(3, 1012, 3036)

In [54]:
napari.view_image(montage_np)

Viewer(camera=Camera(center=(0.0, np.float64(505.5), np.float64(1517.5)), zoom=np.float64(0.18774703557312253), angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True, orientation=(<DepthAxisOrientation.TOWARDS: 'towards'>, <VerticalAxisOrientation.DOWN: 'down'>, <HorizontalAxisOrientation.RIGHT: 'right'>)), cursor=Cursor(position=(np.float64(1.0), 1.0, 0.0), scaled=True, style=<CursorStyle.STANDARD: 'standard'>, size=1.0), dims=Dims(ndim=3, ndisplay=2, order=(0, 1, 2), axis_labels=('0', '1', '2'), rollable=(True, True, True), range=(RangeTuple(start=np.float64(0.0), stop=np.float64(2.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(1011.0), step=np.float64(1.0)), RangeTuple(start=np.float64(0.0), stop=np.float64(3035.0), step=np.float64(1.0))), margin_left=(0.0, 0.0, 0.0), margin_right=(0.0, 0.0, 0.0), point=(np.float64(1.0), np.float64(505.0), np.float64(1517.0)), last_used=0), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False, spa

In [15]:
viewer = napari.view_image(average_project)